<a href="https://colab.research.google.com/github/Decoding-Data-Science/zain/blob/main/Copy_of_crewai_26th.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.15.2 which is incompatible.


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [ ]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [ ]:
import os

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [ ]:
import os
from google.colab import userdata

# Retrieve API key from Colab secrets and set it in environment variables
os.environ["OPENAI_API_KEY"] = userdata.get('openai')

In [ ]:
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

In [ ]:
# Set up API keys - enter from https://serper.dev/billing
os.environ["SERPER_API_KEY"] = "" # serper.dev API key

# Instantiate tools
docs_tool = DirectoryReadTool(directory='./blog-posts')
file_tool = FileReadTool()
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."    ,
    tools=[search_tool, web_rag_tool,docs_tool,file_tool],
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic in 1500 words: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.1500 words",
    agent=writer,
)

### Task: Edit

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    planning=True,
    verbose=4
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
result = crew.kickoff(inputs={"topic": "Model Context Protocol"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Model Context Protocol.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
First, I need to understand what the Model Context Protocol is and what it constitutes. I'll start by searching for this topic on the internet.

Action: 
Search the internet

Action Input: 
{"search_query": "Model Context Protocol"}
 


Search results: Title: Introducing the Model Context Protocol - Anthropic
Link: https://www.anthropic.com/news/model-context-protocol
Snippet: The Model Context Protocol is an open standard that enables developers to build secure, two-way connections between their data sources and AI- ...
---
Title: Model Context Protocol:

- Display the results of your execution as markdown in the notebook.

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Understanding the Model Context Protocol: Developments, Main Contributors, and Insights

## I. Introduction

The Model Context Protocol (MCP) is a vital element of advanced artificial intelligence. It acts as a conduit, facilitating communication between AI developers and AI agents, and refining the interaction process. The significance of MCP in AI applications is profound, as it provides a more efficient and potent method for AI agents to comprehend and react to developers' commands.

## II. Recent Developments

The landscape of AI development is experiencing an upswing in the adoption of MCP. This can be attributed partly to an increase in tools that support MCP, rendering it more accessible to developers. The ecosystem of MCP is in a state of constant evolution, with fresh features and enhancements being introduced regularly, thereby accentuating its pivotal role in AI applications.

## III. Main Contributors

Anthropic, a forefront AI research company, has had a considerable influence in developing and maintaining MCP. Tech behemoths like Microsoft and Cisco, along with AI-centric organizations such as OpenAI, have also embraced MCP, thereby aiding its growth and expansion. Their participation highlights the broad acceptance and the potential of MCP in the AI industry.

## IV. Significant News

Numerous developments in the MCP sphere have recently caught attention. Anthropic took a visionary step by making MCP open source, thus encouraging a broader adoption rate. Docker, a notable player in the tech field, has publicized plans to enhance MCP security. Google has declared that it will incorporate MCP into all its products, thereby expanding its impact. Microsoft has also introduced MCP in its Copilot Studio, further advocating its importance. Additionally, an updated version of the MCP spec has been launched, offering more sophisticated features and capacities for AI developers.

## V. Intended Readers

This blog post is particularly beneficial for AI developers who wish to stay abreast of the latest trends in the field. Companies considering AI integration can also derive valuable insights about the adoption of MCP. Tech enthusiasts eager to understand the latest progressions in AI will also find this post insightful.

## VI. Call to Action

The progressions in MCP are occurring at a breakneck pace. It is vital for AI developers, tech companies, and enthusiasts to keep up with these advancements. If you are an AI developer or a company contemplating AI integration, it's high time to explore the possibility of incorporating MCP into your applications. The benefits are palpable, and the future of MCP seems promising.

Remember to follow our blog for more insights into AI development trends. Let's together navigate the exhilarating world of artificial intelligence and MCP!

Sources: Official documentation of MCP, Github page of MCP, Anthropic's website, OpenAI's website, Various articles on Medium and other tech blogs.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "AI Legislation"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on AI Legislation.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
First, I need to determine the latest trends, key players, and noteworthy news on AI Legislation. I will search the internet for this information.

Action: 
Search the internet

Action Input: 
{"search_query": "latest trends in AI Legislation"}
 


Search results: Title: AI Watch: Global regulatory tracker - United States | White & Case LLP
Link: https://www.whitecase.com/insight-our-thinking/ai-watch-global-regulatory-tracker-united-states
Snippet: On May 17, 2024, Colorado enacted the first comprehensive US AI legislation, the Colorado AI Act. · The Colorado AI

In [ ]:
from IPython.display import Markdown
Markdown(result)

# Understanding the Changing Terrain of AI Legislation

Artificial Intelligence (AI) has become a cornerstone of our daily lives, emphasizing the need for AI Legislation. The implications of AI are extensive and intricate, necessitating broad legal frameworks to ensure ethical and secure use. In this ever-changing environment, staying informed about the current trends and key players involved is crucial.

## Crucial Trends in AI Legislation 

A fresh wave of extensive AI legislation is emerging globally, with laws such as the Colorado AI Act at the forefront. These legislations target to regulate AI applications, like chatbots, ensuring they operate transparently and ethically. There is an increasing demand for a moratorium on powerful AI systems due to apprehensions about their impact on privacy and security.

The trend is also leaning towards sector-specific AI regulations, acknowledging that AI's influence varies across industries. State legislations are advocating transparency and risk-based approaches, striving for a balance between innovation and regulation.

## Major Players in AI Legislation 

The European Union (EU) is a significant force in AI regulation, steering the global dialogue on AI legislation. However, standalone regions are also crafting their frameworks, reflecting their unique needs and considerations.

Within the AI value chain, various actors such as providers, deployers, product manufacturers, importers, and distributors play vital roles in shaping AI legislation. Moreover, the Senate Committee on Commerce and Science is deeply involved in AI-related legislation, underlining the importance of AI in national security and economic prosperity.

## Significant Updates in AI Legislation 

Recently, we've witnessed the adoption of pioneering AI legislation centred on consumer protections. AI regulatory bills are progressing, mirroring the expanding role of AI in our lives. However, not all legislative efforts see the light of day, as indicated by the rejection of the High-Risk Artificial Intelligence Developer and Deployer Act in Virginia.

Legislative bodies are contemplating hundreds of AI-related proposals, indicating the increasing acknowledgement of AI's influence. South Korea has recently introduced a new AI Framework Act, signaling the global move towards comprehensive AI legislation. During the Trump administration, there were noticeable shifts in AI policies, reflecting the evolving position on AI legislation.

## Wrapping Up and Call to Action 

Grasping and staying updated with AI legislation is essential for all stakeholders. As AI becomes a fundamental part of our lives, it's vital to stay educated about new developments in AI Legislation. By doing so, we can ensure the ethical, secure, and beneficial use of AI, crafting a future where technology serves humanity optimally.

Remember, staying ahead in AI Legislation isn't purely about compliance; it's about comprehending the direction of technological innovation and its impact on society and business. So, keep learning, stay current, and be part of the dialogue shaping the future of AI Legislation.

Sources:
[Whitecase](https://www.whitecase.com/insight-our-thinking/ai-watch-global-regulatory-tracker-united-states)
[Insideprivacy](https://www.insideprivacy.com/artificial-intelligence/blog-post-state-legislatures-consider-new-wave-of-2025-ai-legislation/)
[Ncsl](https://www.ncsl.org/technology-and-communication/artificial-intelligence-2024-legislation)
[Ey](https://www.ey.com/en_us/insights/ai/how-to-navigate-global-trends-in-artificial-intelligence-regulation)
[Dataguidance](https://www.dataguidance.com/opinion/usa-common-themes-2024-state-ai-legislation-and)
[Dentons](https://www.dentons.com/en/insights/articles/2025/january/10/ai-trends-for-2025-ai-regulation-governance-and-ethics)
[Cdt](https://cdt.org/insights/regulating-public-sector-ai-emerging-trends-in-state-legislation/)
[Natlawreview](https://natlawreview.com/article/emerging-trends-ai-governance-insights-state-level-regulations-enacted-2024)
[Mwe](https://www.mwe.com/insights/ai-and-the-law-emerging-trends-in-enforcement/)
[Sig](https://www.softwareimprovementgroup.com/us-ai-legislation-overview/)
[Dlapiper](https://www.dlapiper.com/en-us/insights/publications/ai-outlook/2024/major-ai-legislation-advances-in-senate-key-points)
[Bakerdonelson](https://www.bakerdonelson.com/whos-who-under-the-eu-ai-act-spotlight-on-key-actors)
[Kempitlaw](https://kempitlaw.com/insights/ai-act-in-focus-key-players-in-ai-providers/)
[Ncsl](https://www.ncsl.org/resources/details/3-trends-emerge-as-ai-legislation-gains-momentum)
[Cbia](https://www.cbia.com/news/issues-policies/lawmakers-advance-ai-bills/)
[Thefire](https://www.thefire.org/news/ai-new-laws-govern-it-dont-have-be)
[Bsa](https://www.bsa.org/news-events/news/2025-state-ai-wave-building-after-700-bills-in-2024)
[Statescoop](https://statescoop.com/ai-legislation-state-regulation-2024/)
[Insideglobaltech](https://www.insideglobaltech.com/2025/02/21/state-legislatures-consider-new-wave-of-2025-ai-legislation/)
[Fpf](https://fpf.org/blog/south-koreas-new-ai-framework-act-a-balancing-act-between-innovation-and-regulation/)
[Whitehouse](https://www.whitehouse.gov/fact-sheets/2025/01/fact-sheet-president-donald-j-trump-takes-action-to-enhance-americas-ai-leadership/)

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.5.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.15.2 which is incompatible.


In [ ]:
import gradio as gr
from crewai import Agent, Task, Crew

# Define Agents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover intricate insights into the given topic",
    backstory="A seasoned expert in extracting valuable information and providing detailed analysis on any given subject.",
    allow_delegation=False
)

writer = Agent(
    role="Content Writer",
    goal="Craft engaging and informative articles",
    backstory="An experienced content writer who can turn complex research into easy-to-understand and engaging articles.",
    allow_delegation=False
)

editor = Agent(
    role="Content Editor",
    goal="Ensure articles are polished, coherent, and engaging",
    backstory="An expert editor with a sharp eye for clarity, grammar, and flow, capable of turning drafts into publication-ready articles.",
    allow_delegation=False
)

# Function to setup tasks and run CrewAI
def run_crewai_article_writer(topic):
    # Define tasks
    research_task = Task(
        description=f"Conduct in-depth research on the topic: {topic} and provide a structured summary.",
        expected_output="A structured and detailed research summary covering important aspects, recent developments, and insights related to the topic.",
        agent=researcher
    )

    writing_task = Task(
        description=f"Using the research, write a full-length article on the topic: {topic} in 1000 words give various sub section",
        expected_output="An engaging, detailed, informative, and coherent article suitable for online publication.",
        agent=writer
    )

    editing_task = Task(
        description=f"Review and edit the article on {topic} for clarity, grammar, flow, and engagement.",
        expected_output="A polished, clear, and professionally edited final version of the article.",
        agent=editor
    )

    # Setup Crew
    crew = Crew(
        agents=[researcher, writer, editor],
        tasks=[research_task, writing_task, editing_task],
        verbose=True  # Set to False if you want less console output
    )

    # Execute
    result = crew.kickoff()
    return result

# Gradio Interface
def generate_article(topic):
    return run_crewai_article_writer(topic)

iface = gr.Interface(
    fn=generate_article,
    inputs=gr.Textbox(lines=2, placeholder="Enter a topic for the article..."),
    outputs=gr.Textbox(lines=20, label="Generated Article"),
    title="DDS CrewAI Article Writer with Researcher, Writer, and Editor",
    description="Enter a topic and let CrewAI agents research, write, and edit a complete article!"
)

if __name__ == "__main__":
    iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ac7f3e33493f0b5728.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
import os
from google.colab import files
import shutil

# Step 1: Create target directory
upload_dir = "blog-posts"
os.makedirs(upload_dir, exist_ok=True)
print(f"✅ Directory '{upload_dir}' is ready.")

✅ Directory 'blog-posts' is ready.


In [2]:


# Step 2: Upload file(s)
uploaded_files = files.upload()

# Step 3: Move uploaded files to the target folder
for filename in uploaded_files.keys():
    shutil.move(filename, os.path.join(upload_dir, filename))
    print(f"📥 Moved '{filename}' to '{upload_dir}/'")

print("✅ All files uploaded and saved.")


Saving Social Media Guidelines for UAE gov 2021 En.pdf to Social Media Guidelines for UAE gov 2021 En.pdf
📥 Moved 'Social Media Guidelines for UAE gov 2021 En.pdf' to 'blog-posts/'
✅ All files uploaded and saved.


In [ ]:
# Already executed in Colab
docs_tool = DirectoryReadTool(directory='blog-posts')


In [ ]:
from crewai_tools import DirectoryReadTool

# Tool setup (ensure directory exists)
docs_tool = DirectoryReadTool(directory='blog-posts')

# Agent using uploaded documents
researcher = Agent(
    role="Senior Research Specialist",
    goal="Uncover insights using uploaded documents and online sources",
    backstory="You're an expert in using both uploaded files and web search to produce comprehensive insights.",
    tools=[docs_tool],  # ← Enable document access
    allow_delegation=False
)


In [ ]:
crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    verbose=True
)

result = crew.kickoff(inputs={"topic": "AI Regulation"})
print(result)
